### Extra special command line stuffs

In [1]:
# !pip3 install ipywidgets
# !pip3 install jupyternotify
# !pip3 install jupyterthemes
# !pip3 install autopep8
# !pip3 install -U jupyter
# !pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user

# !jupyter nbextension enable --py widgetsnbextension
# !jupyter contrib nbextension install --sys-prefix

# !jt -t chesterish -f fira -fs 10 -nf ptsans -nfs 11 -N -kl -cursw 2 -cursc r -cellw 95% -T 


### Regular python program stuff

In [1]:
# standard funtime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.style
matplotlib.style.use('dark_background')

# special funtime
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core import interactiveshell as InteractiveShell
import jupyternotify



In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, make_scorer

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config IPCompleter.greedy = True
%reload_ext jupyternotify
pd.options.display.max_columns = None
pd.options.display.max_rows = 200
sns.set_palette("bright")
sns.set(style="darkgrid")
sns.set(rc={'figure.figsize':(30,30)})
InteractiveShell.ast_node_interactivity = "all"
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
%autonotify -a 120


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# the train-test split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=42
)

NameError: name 'train_test_split' is not defined

In [ ]:
cross_validate(estimator=LinearRegression(),
                X=X_train2,
                y=y_train2,
                scoring=mse,
                return_estimator=True)

In [ ]:
# a basic plot... thing

fig, ax = plt.subplots()

X = np.linspace(0, 310, 600)
y = lr2.coef_ * X + lr2.intercept_

ax.scatter(df['pred'], df['target'])
ax.plot(X, y)
ax.set_title(f"""The best-fit line when scaling after splitting is
    {round(lr2.coef_[0])}X + {round(lr2.intercept_)}""");

In [ ]:
#logodds
def log_interpret(coef_):
    return np.exp(coef_) / (1 + np.exp(coef_))

In [ ]:
#heatmap with mask
matrix = np.triu(df.corr())

sns.heatmap(df.corr(), annot=True, mask=matrix)

In [ ]:
class ModelWithCV():
    '''Structure to save the model and more easily see its crossvalidation'''
    
    def __init__(self, model, model_name, X, y, cv_now=True):
        self.model = model
        self.name = model_name
        self.X = X
        self.y = y
        # For CV results
        self.cv_results = None
        self.cv_mean = None
        self.cv_median = None
        self.cv_std = None
        #
        if cv_now:
            self.cross_validate()
        
    def cross_validate(self, X=None, y=None, kfolds=10):
        '''
        Perform cross-validation and return results.
        
        Args: 
          X:
            Optional; Training data to perform CV on. Otherwise use X from object
          y:
            Optional; Training data to perform CV on. Otherwise use y from object
          kfolds:
            Optional; Number of folds for CV (default is 10)  
        '''
        
        cv_X = X if X else self.X
        cv_y = y if y else self.y

        self.cv_results = cross_val_score(self.model, cv_X, cv_y, cv=kfolds)
        self.cv_mean = np.mean(self.cv_results)
        self.cv_median = np.median(self.cv_results)
        self.cv_std = np.std(self.cv_results)

        
    def print_cv_summary(self):
        cv_summary = (
        f'''CV Results for `{self.name}` model:
            {self.cv_mean:.5f} ± {self.cv_std:.5f} accuracy
        ''')
        print(cv_summary)

        
    def plot_cv(self, ax):
        '''
        Plot the cross-validation values using the array of results and given 
        Axis for plotting.
        '''
        ax.set_title(f'CV Results for `{self.name}` Model')
        # Thinner violinplot with higher bw
        sns.violinplot(y=self.cv_results, ax=ax, bw=.4)
        sns.swarmplot(
                y=self.cv_results,
                color='orange',
                size=10,
                alpha= 0.8,
                ax=ax
        )

        return ax

In [ ]:
def add_missing_indicator_columns(X, indicator):
    """
    Helper function for transforming features
    
    For every feature in X, create another feature indicating whether that feature
    is missing. (This doubles the number of columns in X.)
    """
    
    # create a 2D array of True and False values indicating whether a given feature
    # is missing for that row
    missing_array_bool = indicator.transform(X)
    
    # transform into 1 and 0 for modeling
    missing_array_int = missing_array_bool.astype(int)
    
    # helpful for readability but not needed for modeling
    missing_column_names = [col + "_missing" for col in X.columns]
    
    # convert to df so it we can concat with X
    missing_df = pd.DataFrame(missing_array_int, columns=missing_column_names, index=X.index)
    
    return pd.concat([X, missing_df], axis=1)